# This notebook is for competion using LSTM model and Same values

* multiprocess to speed up the process
* get train data
* get result data with category

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import multiprocessing as mp
import pandas as pd
import numpy as np
from functools import partial
import random

from helper_functions import *

In [17]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv').sort_values(by='row_id').reset_index().drop('index', axis=1)
results = pd.read_csv('lstm_sv_result_with_category.csv')

In [18]:
test.head(10)

,row_id,cfips,first_day_of_month
0,10001_2022-11-01,10001,2022-11-01
1,10001_2022-12-01,10001,2022-12-01
2,10001_2023-01-01,10001,2023-01-01
3,10001_2023-02-01,10001,2023-02-01
4,10001_2023-03-01,10001,2023-03-01
5,10001_2023-04-01,10001,2023-04-01
6,10001_2023-05-01,10001,2023-05-01
7,10001_2023-06-01,10001,2023-06-01
8,10003_2022-11-01,10003,2022-11-01
9,10003_2022-12-01,10003,2022-12-01


In [4]:
results.head()

,Country,MAPE,SV_MAPE,Active,category
0,1001,0.732748,1.835232,1417,lstm
1,1003,0.747286,5.230218,13401,lstm
2,1005,1.273616,1.101416,239,SV
3,1007,9.315642,3.574082,220,SV
4,1009,0.766072,2.116541,789,lstm


In [5]:
result = get_score_for_competition(train, results, window_size=9, horizon=8, epoch=500, split_size=0, cfips=1001)

1/1 [==============================] - 0s 285ms/step


In [6]:
result

(1001,
 array([3.3354104, 3.353883 , 3.3726413, 3.3782535, 3.3977926, 3.4205523,
        3.435101 , 3.4592876], dtype=float32))

In [7]:
np.array(result[1]).reshape((-1))

array([3.3354104, 3.353883 , 3.3726413, 3.3782535, 3.3977926, 3.4205523,
       3.435101 , 3.4592876], dtype=float32)

In [8]:
cfips_list = []
forecasts = []

for c in results[:5].Country:
    result = get_score_for_competition(train, results, window_size=9, horizon=8, epoch=500, split_size=0, cfips=c)
    cfips_list.append(result[0])
    forecasts.append(result[1])

1/1 [==============================] - 0s 260ms/step


In [9]:
forecasts

[array([3.3354104, 3.353883 , 3.3726413, 3.3782535, 3.3977926, 3.4205523,
        3.435101 , 3.4592876], dtype=float32),
 array([8.237763, 8.221723, 8.225635, 8.271938, 8.280881, 8.321825,
        8.322697, 8.332918], dtype=float32),
 array([1.2068269, 1.2068269, 1.2068269, 1.2068269, 1.2068269, 1.2068269,
        1.2068269, 1.2068269]),
 array([1.2366498, 1.2366498, 1.2366498, 1.2366498, 1.2366498, 1.2366498,
        1.2366498, 1.2366498]),
 array([1.7833898, 1.7921008, 1.8003149, 1.8043028, 1.8085302, 1.8231107,
        1.8321555, 1.8397001], dtype=float32)]

In [15]:
predictions = np.array(forecasts).reshape((-1))

In [19]:
sample_test = test[:40]
sample_test['microbusiness_density'] = predictions
sample_test = sample_test[['row_id', 'microbusiness_density']]
sample_test

C:\Users\Masa\AppData\Local\Temp\ipykernel_25204\2835970789.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_test['microbusiness_density'] = predictions


,row_id,microbusiness_density
0,10001_2022-11-01,3.335410
1,10001_2022-12-01,3.353883
2,10001_2023-01-01,3.372641
3,10001_2023-02-01,3.378253
4,10001_2023-03-01,3.397793
5,10001_2023-04-01,3.420552
6,10001_2023-05-01,3.435101
7,10001_2023-06-01,3.459288
8,10003_2022-11-01,8.237763
9,10003_2022-12-01,8.221723


## Muitiprocess to do all of the data

In [20]:
%%time
if __name__ == '__main__':
    HORIZON = 8
    WINDOW_SIZE = 9

    train = pd.read_csv('train.csv')
    IDS = train.cfips.unique()

    window_size = WINDOW_SIZE
    horizon = HORIZON
    epoch = 500
    split_size = 0
    
    cfips_list = []
    forecasts = []

    pool = mp.Pool(16)

    func = partial(get_score_for_competition, train, results, window_size, horizon, epoch, split_size)
    results = pool.map(func, IDS)
    pool.close()
    pool.join()
    
    for i in range(len(results)):
        c = results[i][0]
        cfips_list.append(c)
        forecasts.append(results[i][1])
    
    test['microbusiness_density'] = np.array(forecasts).reshape((-1))
    submission = test[['row_id', 'microbusiness_density']]

CPU times: total: 6.22 s
Wall time: 59min 45s


In [21]:
submission.head(10)

,row_id,microbusiness_density
0,10001_2022-11-01,3.335410
1,10001_2022-12-01,3.353883
2,10001_2023-01-01,3.372641
3,10001_2023-02-01,3.378253
4,10001_2023-03-01,3.397793
5,10001_2023-04-01,3.420552
6,10001_2023-05-01,3.435101
7,10001_2023-06-01,3.459288
8,10003_2022-11-01,8.237763
9,10003_2022-12-01,8.221723


In [22]:
submission.to_csv('submission_1.csv', index=False)